In [1]:
IPCC = "ipcc"
OIL186 = "oil186"
CLIMATE100 = "climate100"
# PROJECT = IPCC
PROJECT = OIL186
PROJECT = CLIMATE100
# DEBUG = True
DEBUG = False
LOCAL = True
HOME_PROJECTS = "/Users/pm286/projects"
NOTEBOOK = "/content/{PROJECT}"
if PROJECT == OIL186:
    MAKE_DICT = True
    GITHUB_SITE = "https://raw.githubusercontent.com"
    REPO = "petermr/CEVOpen"
    BLOB = "master"
    PROJ_TREE = "searches/oil186"
    HTML = 'scholarly.html'
    LOCAL_DIR = f"{HOME_PROJECTS}/CEVOpen/searches/oil186"
elif PROJECT == CLIMATE100:
    MAKE_DICT = True
    GITHUB_SITE = "https://raw.githubusercontent.com"
    REPO = "petermr/climate"
    BLOB = "master"
    PROJ_TREE = "searches/oil186"
    HTML = 'scholarly.html'
    LOCAL_DIR = f"{HOME_PROJECTS}/climate"
    DICT_DIR = LOCAL_DIR
    DICT_NAME = "climate_terms"

elif PROJECT == IPCC:
    MAKE_DICT = True
    GITHUB_SITE = "https://raw.githubusercontent.com"
    REPO = "petermr/semanticClimate"
    BLOB = "master"
    PROJ_TREE = "ipcc/ar6/wg3"
    HTML = 'fulltext.flow.html'


INSTALL = False
if INSTALL:

    !pip install wikipedia #to scrape wikipedia for climate-related terms
    !pip install docanalysis # Semantic Climate's tool to search using existing dictionaries and create new ones from scratch
    !pip install wordcloud # to represent search results as Wordclouds
    !docanalysis --help

# Getting started
This notebook should appear in your browser...

This operation loads all the software into Google Colabs automatically

## Install packages

This Colab Notebook is a part of Post Hackthon Pitch to SDG Accelerator, Geneva. Make a copy of it in your Drive and run it. 

- [Presentation Video](https://youtu.be/0MGXic-uEQY): 12 min. video on YouTube
- [Slides (PPT)](https://github.com/petermr/semanticClimate/blob/main/outreach/piaget_hackathon/post_hackathon_pitch_v3.pptx)

In [2]:
# importing all packages
from pprint import pprint
import wikipedia
import urllib
from bs4 import BeautifulSoup # to parse html and pull out text
import re
import requests
import os
from lxml import etree # to create dictionaries 
from collections import Counter
import json
from wordcloud import WordCloud

## Get HTML

[`pyami`](https://github.com/petermr/pyami) converts PDF IPCC Reports into HTMLs that we store in our SemanticClimate GitHub Repository. You can find them, [here](https://github.com/petermr/semanticClimate/tree/main/ipcc/ar6/wg3).

In [3]:
html_dict = dict()
if PROJECT == IPCC:
#     python dictionary with links to raw IPCC reports HTML
    CTREE_LIST = [
        "Chapter01",
        "Chapter04",
        "Chapter06",
        "Chapter07",
        "Chapter15",
        "Chapter16",
    ]
    for CTREE in CTREE_LIST:
        url = f"{GITHUB}/{REPO}/{BLOB}/{PROJ_TREE}/{CTREE}/{HTML}"
        html_dict[f"{CTREE}"] = url
        printf(url)
            
        
#         "chap04":"https://raw.githubusercontent.com/petermr/semanticClimate/main/ipcc/ar6/wg3/Chapter04/fulltext.flow.html",
#         "chap06":"https://raw.githubusercontent.com/petermr/semanticClimate/main/ipcc/ar6/wg3/Chapter06/fulltext.flow.html",
#         "chap07":"https://raw.githubusercontent.com/petermr/semanticClimate/main/ipcc/ar6/wg3/Chapter07/fulltext.flow.html",
#         "chap15":"https://raw.githubusercontent.com/petermr/semanticClimate/main/ipcc/ar6/wg3/Chapter15/fulltext.flow.html",
#         "chap16":"https://raw.githubusercontent.com/petermr/semanticClimate/main/ipcc/ar6/wg3/Chapter16/fulltext.flow.html"
    
elif PROJECT == OIL186:
    CTREE_LIST = [
                "PMC4391421", "PMC5080681",
                "PMC5132230", "PMC5203915","PMC5237462", "PMC5248495","PMC5282690", "PMC5307246","PMC5307902", "PMC5324201",
                "PMC5330108", "PMC5344628","PMC5364420", "PMC5393100","PMC5397855","PMC5411863","PMC5412227", "PMC5423258",
                "PMC5426739","PMC5427463","PMC5448358","PMC5454990","PMC5485486","PMC5486035","PMC5497343","PMC5507808","PMC5524814",
                "PMC5527698","PMC5535876","PMC5543433","PMC5546729","PMC5551175","PMC5568258","PMC5569441","PMC5575638",
                "PMC5577677","PMC5585972","PMC5590060","PMC5590062","PMC5590063","PMC5590065","PMC5590066","PMC5590067",
                "PMC5590070","PMC5592951","PMC5597067","PMC5602041","PMC5602841","PMC5603114","PMC5613177","PMC5615139",
                "PMC5615285","PMC5620597","PMC5622382","PMC5622390","PMC5622397","PMC5622398","PMC5622401","PMC5622403",
                "PMC5625792","PMC5641611","PMC5651092","PMC5653886","PMC5660901","PMC5661929","PMC5668225","PMC5669080","PMC5669111",
                "PMC5674267","PMC5684592","PMC5694497","PMC5694587","PMC5694611","PMC5694875","PMC5694991","PMC5699893","PMC5702407","PMC5702920",
                "PMC5717781","PMC5723952","PMC5725564","PMC5735349","PMC5736702","PMC5742650","PMC5745743","PMC5746745","PMC5747963","PMC5748641","PMC5750594",
                "PMC5750605","PMC5750654","PMC5751248","PMC5761127","PMC5772139","PMC5778200","PMC5778779",
                "PMC5788217","PMC5789270","PMC5789316","PMC5794096","PMC5795983","PMC5797122","PMC5806308","PMC5807769","PMC5811758","PMC5813356",
                "PMC5822514","PMC5830750","PMC5838999","PMC5842484","PMC5846372","PMC5848570","PMC5849894","PMC5849899",
                "PMC5849928","PMC5852288","PMC5855832","PMC5858069","PMC5858457","PMC5859817","PMC5867545","PMC5867556",
                "PMC5871051","PMC5871294","PMC5872285","PMC5872290","PMC5874608","PMC5876267","PMC5876298","PMC5877547",
                "PMC5878871","PMC5879832","PMC5884000","PMC5884125","PMC5885327","PMC5886561","PMC5896384","PMC5896386",
                "PMC5897738","PMC5901951","PMC5902937","PMC5905184","PMC5905380","PMC5905578","PMC5909600","PMC5918559",
                "PMC5919639","PMC5920421","PMC5920425","PMC5921405","PMC5923693","PMC5925846","PMC5933010","PMC5933022",
                "PMC5933509","PMC5933692","PMC5937097","PMC5937106","PMC5938542","PMC5940754","PMC5945564","PMC5946457",
                "PMC5947909","PMC5952513","PMC5954905","PMC5956054","PMC5957362","PMC5958151","PMC5958191","PMC5960541",
                "PMC5960548","PMC5961776","PMC5963643","PMC5964621","PMC5970210","PMC5974043","PMC5977410","PMC5978029",
                "PMC5985564","PMC5993771","PMC5997812","PMC6006875","PMC6011056","PMC6011059","PMC6011244","PMC6015887"
                  ]
    
elif PROJECT == CLIMATE100:
    CTREE_LIST = [
                "PMC4391421", "PMC5080681",
    ]
else:
    print(f"BAD project {PROJECT}")

if MAKE_DICT:
    """https://raw.githubusercontent.com/petermr/CEVOpen/master/searches/oil186/PMC4391421/scholarly.html"""
    """make the dict"""
    for CTREE in CTREE_LIST:
        html_dict[CTREE] = f"{GITHUB_SITE}/{REPO}/{BLOB}/{PROJ_TREE}/{CTREE}/{HTML}"
        if DEBUG:
            print(f"url: {html_dict[CTREE]}")
  
    # debug
    if DEBUG:
        for item in list(html_dict.items())[:3]:
            print(f" {item}")
        if PROJECT == PMCID:
            pmcid = "PMC4391421"
            html_url = html_dict[pmcid]
            assert html_url, f"test html {pmcid}"
            # debug
            f = urllib.request.urlopen(html_url)
            content = f.read().decode("utf-8")
            print(f"content {content}")


In [4]:
# write raw HTML to sections folder

# Part of this code comes from Ananya Singha's notebook
for key,value in html_dict.items():
    SUB_FOLDER = f"{PROJECT}/{key}/sections/"
    if not os.path.exists(SUB_FOLDER):
       os.makedirs(SUB_FOLDER)
    page = requests.get(value)
    with open(SUB_FOLDER + HTML , 'wb+') as f:
        f.write(page.content)

## Extract abbreviations using [`docanalysis`](https://github.com/petermr/docanalysis) and write to an ami-dictionary

In [5]:
!docanalysis --project_name ipcc --extract_abb all_chap --output abb.csv --make_json all_abb.json --search_html

/opt/anaconda3/lib/python3.8/site-packages/_distutils_hack/__init__.py:36: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
100%|█████████████████████████████████████████| 186/186 [00:07<00:00, 25.49it/s]
INFO: Found 1674 sentences in the section(s).
0it [00:00, ?it/s]
INFO: wrote all abbreviations to ami dict /Users/pm286/projects/climate/ipcc/all_chap.xml
INFO: wrote output to /Users/pm286/projects/climate/ipcc/abb.csv
INFO: wrote JSON output to /Users/pm286/projects/climate/ipcc/all_abb.json


## Example Output (ami-dict)
```
<entry name="VRE" term="Variable Renewable Energy" wikidataID="['//www.wikidata.org/wiki/Q7915732']"/>
  <entry name="SDGs" term="sustainable development goals" wikidataID="['//www.wikidata.org/wiki/Q7649586', '//www.wikidata.org/wiki/Q22547988', '//www.wikidata.org/wiki/Q30252260', '//www.wikidata.org/wiki/Q104759743', '//www.wikidata.org/wiki/Q111643133', '//www.wikidata.org/wiki/Q109354788', '//www.wikidata.org/wiki/Q104856926']"/>
  <entry name="TPES" term="total primary energy  supply" wikidataID="[]"/>
  <entry name="TFC" term="total final energy consumption" wikidataID="[]"/>
  <entry name="CSP" term="Concentrating  solar  power" wikidataID="[]"/>
  <entry name="LIBs" term="lithium-ion batteries" wikidataID="['//www.wikidata.org/wiki/Q106988181', '//www.wikidata.org/wiki/Q56157763', '//www.wikidata.org/wiki/Q57959688', '//www.wikidata.org/wiki/Q56157653']"/>
```

## Example Output (as JSON)

```
    "31": {
        "file_path": "/content/ipcc_2/chap16/sections/fulltext.flow.html",
        "sentence": "However, there is a growing body of evidence from developing  countries and small island developing states (SIDS) {16.4, 16.4.4.3, 16.4.4.4, 16.5, 16.7}.",
        "section": "HTML",
        "abb": {
            "SIDS": "small island developing states"
        }
    },
    "57": {
        "file_path": "/content/ipcc_2/chap16/sections/fulltext.flow.html",
        "sentence": "Sustainable  solutions  require  adoption  and  mainstreaming  of  locally  novel  technologies  that  can  meet  local  needs,  and  simultaneously  address  the  Sustainable  Development Goals (SDGs).",
        "section": "HTML",
        "abb": {
            "SDGs": "Sustainable  Development Goals"
        }
    },
    "74": {
        "file_path": "/content/ipcc_2/chap16/sections/fulltext.flow.html",
        "sentence": "Furthermore,  methodological  developments  to  improve  the  ability  of  Integrated  Assessment  Models (IAMs) to capture energy innovation system dynamics, and the relevant institutions and  policies  (including  design  and  implementation),  would  allow  for  more  realistic  assessment.",
        "section": "HTML",
        "abb": {
            "IAMs": "Integrated  Assessment  Models"
        }
    },
    "146": {
        "file_path": "/content/ipcc_2/chap16/sections/fulltext.flow.html",
        "sentence": "The  section  also  describes the different phases of innovation and metrics, such as the widely used but also criticized  technology readiness levels (TRLs).",
        "section": "HTML",
        "abb": {
            "TRLs": "technology readiness levels"
        }
    }
```

## Make dictionary by extracting terms from Wikipedia

In Part 1, we created dictionary by extracting information from the reports itself. These types of dictionaries are used to index HTML. 

In this part, Part 2, we'll show you how to create dictionaries from other sources and databases. We can use these dictionaries to search the HTML reports. 

In [6]:
def get_raw_wikipedia_html(title_of_page):
  raw_html = wikipedia.WikipediaPage(title_of_page).html()
  return raw_html

def get_terms(raw_html, find_element = "dt"):
  soup = BeautifulSoup(raw_html, 'html.parser')
  climate_term = []
  black_list = ['climate', 'climate change']
  for term in soup.find_all(find_element):
      our_term = term.text.replace('\n', ' ')
      clean_term = re.sub(r'\([^()]*\)', '', our_term)
      if clean_term not in black_list:
        climate_term.append(clean_term)
  return climate_term

def write_to_dict(climate_term, title):
    dictionary_element = etree.Element("dictionary")
    dictionary_element.attrib['title'] = title
    for term in climate_term:
        entry_element = etree.SubElement(dictionary_element, "entry")
        entry_element.attrib['term'] = term
    xml_dict = etree.tostring(dictionary_element, pretty_print=True).decode('utf-8')

    with open(f'{title}.xml', mode='w', encoding='utf-8') as f:
        f.write(xml_dict)

In [7]:
print(f"pwd=", end="")
!pwd


pwd=/Users/pm286/projects/climate


In [8]:
# Good starting point to create climate terms dictionary: https://en.wikipedia.org/wiki/Glossary_of_climate_change
title_of_page = "Glossary of climate change" 
raw_html = get_raw_wikipedia_html(title_of_page)
climate_term = get_terms(raw_html)
write_to_dict(climate_term, 'climate_terms')

## Search reports for the terms from Climate Terms ami-dictionary

In [9]:
!docanalysis --project_name ipcc --search_html --dictionary {DICT_DIR}/{DICT_NAME}.xml --output {DICT_NAME}.csv --make_json {DICT_NAME}.json

/opt/anaconda3/lib/python3.8/site-packages/_distutils_hack/__init__.py:36: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
100%|█████████████████████████████████████████| 186/186 [00:06<00:00, 26.57it/s]
INFO: Found 1674 sentences in the section(s).
INFO: getting terms from /Users/pm286/projects/climate/climate_terms.xml
100%|█████████████████████████████████████| 1674/1674 [00:01<00:00, 1421.47it/s]
INFO: wrote output to /Users/pm286/projects/climate/ipcc/climate_terms.csv
INFO: wrote JSON output to /Users/pm286/projects/climate/ipcc/climate_terms.json


In [10]:
ls -l {DICT_DIR}/{DICT_NAME}.xml 

-rw-r--r--  1 pm286  staff  5796 13 Aug 22:19 /Users/pm286/projects/climate/climate_terms.xml


In [11]:
# finding out the most common climate-related terms mentioned in the reports
def get_hit_counts(path):
    with open(path, 'r') as f:
        json_dict = json.load(f)

    term_list = []
    for key, value in json_dict.items():
      term_list.extend(value["0"])
    final_list = [item for sublist in term_list for item in sublist]
    return (Counter(final_list))

In [12]:
# generating wordcloud from search hits
def generate_wordcloud_and_save(hits_dictionary, png_file):
    wc = WordCloud(background_color='white', width = 700, height=300, margin=1)
    wc.fit_words(hits_dictionary)
    wc.to_file(png_file)

In [13]:
json_file = '/content/ipcc/{DICT_NAME}.json'
png_file = f'{DICT_NAME}.png'
climate_terms_hits = get_hit_counts(json_file)
pprint(climate_terms_hits)
generate_wordcloud_and_save(climate_terms_hits, png_file)

FileNotFoundError: [Errno 2] No such file or directory: '/content/ipcc/climate_terms.json'

In [ ]:
from IPython.display import Image
Image('/content/climate_terms.png')

## Search reports for countries using existing dictionary

In [ ]:
!docanalysis --project_name ipcc --search_html --dictionary COUNTRY --output country.csv --make_json country.json

In [ ]:
COUNTRY_PATH = '/content/ipcc/country.json'
COUNTRY_FILE_NAME = '/content/country.png'
country_hits = get_hit_counts(COUNTRY_PATH)
pprint(country_hits)
generate_wordcloud(country_hits, COUNTRY_FILE_NAME)

In [ ]:
from IPython.display import Image
Image(COUNTRY_FILE_NAME)

In [ ]:
!docanalysis --project_name ipcc --search_html --dictionary COUNTRY /content/climate_terms.xml --output country_climate_terms.csv --make_json country_climate_terms.json

```
    "9895": {
        "file_path": "/content/ipcc/chap16/sections/fulltext.flow.html",
        "sentence": "Furthermore, existing interests, such as fossil fuel production, may also impede the  deployment  of  low-emission  technologies,  as  highlighted  in  case  studies  of  Vietnam  and  Indonesia  (Dorband  et  al.",
        "section": "HTML",
        "0": [
            [
                "Vietnam",
                "Indonesia"
            ]
        ],
        "0_span": [
            [
                [
                    171,
                    178
                ],
                [
                    185,
                    194
                ]
            ]
        ],
        "weight_0": 2,
        "1": [
            [
                "fossil fuel"
            ]
        ],
        "1_span": [
            [
                [
                    41,
                    52
                ]
            ]
        ],
        "weight_1": 1
    },       
```